In [0]:
import tensorflow as tf
import keras as ks
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [0]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout

In [28]:
classifier = Sequential()

# Step 1 - Convolution
classifier.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))

# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))
#classifier.add(Dropout(p=0.5))

# Adding a second convolutional layer
classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Dropout(p=0.5))

# Step 3 - Flattening
classifier.add(Flatten())

# Step 4 - Full connection
classifier.add(Dense(units = 512, activation = 'relu'))
classifier.add(Dropout(p=0.5))
classifier.add(Dense(units = 1, activation = 'sigmoid'))

# Compiling the CNN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.5)`
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.5)`


In [25]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# Part 2 - Fitting the CNN to the images

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [30]:

training_set = train_datagen.flow_from_directory('/content/drive/My Drive/truckcar/train',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

test_set = test_datagen.flow_from_directory('/content/drive/My Drive/truckcar/test',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

classifier.fit_generator(training_set,
                         steps_per_epoch = 628 ,
                         epochs = 25,
                         validation_data = test_set,
                         validation_steps = 160)


Found 628 images belonging to 2 classes.
Found 160 images belonging to 2 classes.
Epoch 1/25
628/628 [==============================] - 176s 280ms/step - loss: 0.5750 - acc: 0.6774 - val_loss: 0.8684 - val_acc: 0.5938
Epoch 2/25
628/628 [==============================] - 171s 272ms/step - loss: 0.2988 - acc: 0.8709 - val_loss: 1.2527 - val_acc: 0.5687
Epoch 3/25
628/628 [==============================] - 172s 273ms/step - loss: 0.1366 - acc: 0.9492 - val_loss: 1.3569 - val_acc: 0.6312
Epoch 4/25
628/628 [==============================] - 170s 271ms/step - loss: 0.0716 - acc: 0.9758 - val_loss: 1.8222 - val_acc: 0.6000
Epoch 5/25
628/628 [==============================] - 171s 273ms/step - loss: 0.0545 - acc: 0.9805 - val_loss: 1.5360 - val_acc: 0.6000
Epoch 6/25
628/628 [==============================] - 172s 273ms/step - loss: 0.0394 - acc: 0.9866 - val_loss: 1.8629 - val_acc: 0.6062
Epoch 7/25
628/628 [==============================] - 170s 271ms/step - loss: 0.0384 - acc: 0.9885 - v

In [0]:
##Comments on Accuracy
from keras.preprocessing import image
test_image = image.load_img('dataset/single_prediction/cat_or_truck_1.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = classifier.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
    prediction = 'dog'
else:
    prediction = 'cat'
    


### Training Accuracy comes to 99.4% and Validation Accuracy comes to 61%. These results are obtained after 25 epochs. The huge difference in training and validation accuracy leads to overfitting.

Various variations of the models were tried for Car and Truck.
1. Basic Model of 2 layers
2. Regularization was applied by using Dropout layers but it still did not lead to an increase in accuracy.

Hence my final model is the one which has two layers and certain dropout layers.